In [1]:
import cv2
import numpy as np
import mediapipe as mp
import serial
import time

arduino=serial.Serial('COM5',9600)
time.sleep(2)

# Variables globales
background = None
hand = None
frames_elapsed = 0
screenshot_counter = 1

# Configuraciones iniciales
FRAME_WIDTH = 700    # Ancho de la ventana
FRAME_HEIGHT = 400   # Alto de la ventana
CALIBRATION_TIME = 50  # Tiempo de calibración en fotogramas
BG_WEIGHT = 0.5      # Peso para el promedio acumulativo del fondo
OBJ_THRESHOLD = 18   # Umbral para la segmentación del objeto

# MediaPipe configuración
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# Clase para gestionar datos de la mano
class HandData:
    fingers = None
    gestureList = []
    
    def __init__(self, top=(0, 0), bottom=(0, 0), left=(0, 0), right=(0, 0), centerX=0):
        self.top = top
        self.bottom = bottom
        self.left = left
        self.right = right
        self.centerX = centerX
        self.prevCenterX = 0
        self.isInFrame = False
        self.isWaving = False

    def update(self, top, bottom, left, right, centerX):
        self.top = top
        self.bottom = bottom
        self.left = left
        self.right = right
        self.prevCenterX = self.centerX
        self.centerX = centerX

    def check_for_waving(self):
        if abs(self.centerX - self.prevCenterX) > 3:
            self.isWaving = True
        else:
            self.isWaving = False

def detect_fingers_mediapipe(frame, hands):
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)
    fingers_counter = "_"

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            height, width, _ = frame.shape
            coordinates_ft = [  # Puntas de los dedos
                (int(hand_landmarks.landmark[i].x * width),
                 int(hand_landmarks.landmark[i].y * height))
                for i in [8, 12, 16, 20]  # Índice, medio, anular, meñique
            ]

            coordinates_palm = [  # Centro de la palma
                (int(hand_landmarks.landmark[i].x * width),
                 int(hand_landmarks.landmark[i].y * height))
                for i in [0, 1, 2, 5, 9, 13, 17]
            ]

            centroid = palm_centroid(coordinates_palm)
            
            # Calcular distancias
            d_centrid_ft = [np.linalg.norm(np.array(centroid) - np.array(pt)) for pt in coordinates_ft]
            fingers_raised = sum([d > 50 for d in d_centrid_ft])  # Límite empírico para dedos levantados
            fingers_counter = str(fingers_raised)

    return fingers_counter


def palm_centroid(coordinates_list):
    coordinates = np.array(coordinates_list)
    centroid = np.mean(coordinates, axis=0)
    centroid = int(centroid[0]), int(centroid[1])
    return centroid

# Definir la región de interés (ROI)
region_top = 0
region_bottom = 220
region_left = FRAME_WIDTH - 220
region_right = FRAME_WIDTH

def get_region(frame):
    region = frame[region_top:region_bottom, region_left:region_right]
    region = cv2.cvtColor(region, cv2.COLOR_BGR2GRAY)
    region = cv2.GaussianBlur(region, (5, 5), 0)
    return region

def get_average(region):
    global background
    if background is None:
        background = region.copy().astype("float")
        return
    cv2.accumulateWeighted(region, background, BG_WEIGHT)

def segment(region):
    global hand
    diff = cv2.absdiff(cv2.convertScaleAbs(background), region)
    _, thresholded_region = cv2.threshold(diff, OBJ_THRESHOLD, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresholded_region, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) == 0:
        if hand:
            hand.isInFrame = False
        return None
    
    max_contour = max(contours, key=cv2.contourArea)
    hull = cv2.convexHull(max_contour)
    
    top = tuple(hull[hull[:, :, 1].argmin()][0])
    bottom = tuple(hull[hull[:, :, 1].argmax()][0])
    left = tuple(hull[hull[:, :, 0].argmin()][0])
    right = tuple(hull[hull[:, :, 0].argmax()][0])
    centerX = int((left[0] + right[0]) / 2)
    
    if not hand:
        hand = HandData(top, bottom, left, right, centerX)
    else:
        hand.update(top, bottom, left, right, centerX)
        hand.check_for_waving()
    hand.isInFrame = True  
    return thresholded_region    

def write_on_image(frame, fingers_detected):
    
    if frames_elapsed < CALIBRATION_TIME:
        text = "Calibrando..."
    elif not hand or not hand.isInFrame:
        text = "Mano no detectada"
    else:
        if hand.isWaving:
            text = "Mano ondeando"
        else:
            text = f"Dedos: {fingers_detected}"

    cv2.putText(frame, text, (10, 20), cv2.FONT_HERSHEY_COMPLEX, 0.4, (0, 0, 0), 2)
    cv2.putText(frame, text, (10, 20), cv2.FONT_HERSHEY_COMPLEX, 0.4, (255, 255, 255), 1)
    cv2.rectangle(frame, (region_left, region_top), (region_right, region_bottom), (255, 255, 255), 2)


    if text == "Dedos: 0":
        arduino.write(b'0')  
    elif text == "Dedos: 1":
        arduino.write(b'1') 
    elif text == "Dedos: 2":
        arduino.write(b'2')  
    elif text == "Dedos: 3":
        arduino.write(b'3') 
    elif text == "Mano ondeando":
        arduino.write(b'4') 
    elif text == "Mano no detectada":
        arduino.write(b'0')  


# Inicialización de la cámara y bucle principal
camera = cv2.VideoCapture(0)
if not camera.isOpened():
    print("No se pudo abrir la cámara")

with mp_hands.Hands(
    model_complexity=1,
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
) as hands:
    while True:
        success, frame = camera.read()
        if not success:
            print("No se pudo leer el cuadro. Fin.")
            break

        frame = cv2.resize(frame, (FRAME_WIDTH, FRAME_HEIGHT))
        frame = cv2.flip(frame, 1)
        region = get_region(frame)

        if frames_elapsed < CALIBRATION_TIME:
            get_average(region)
        else:
            result = segment(region)
            if result is not None:
                cv2.imshow("Mano Segmentada", result)

        fingers_detected = detect_fingers_mediapipe(frame, hands)
        write_on_image(frame, fingers_detected)
        cv2.imshow("Video de la camara", frame)

        frames_elapsed += 1
        key = cv2.waitKey(1) & 0xFF

        if key == ord('s'):
            global screenshot_counter
            filename = f"SS{screenshot_counter}.png"
            cv2.imwrite(filename, frame)
            print(f"Captura de pantalla guardada: {filename}")
            screenshot_counter += 1 
        
        if key == ord('c'):
            break

camera.release()
cv2.destroyAllWindows()
